In [5]:
# Simple class to hold std cell placement attributes
class StdCellInstance:
    def __init__(self, inst_name="", loc=(0,0), orient="N"):
        self.inst_name =inst_name
        self.inst_loc_x = loc[0]
        self.inst_loc_y = loc[1]
        self.orient = orient
    
    def get_place_str(self):
        return(f"{self.inst_name} {self.inst_loc_x} {self.inst_loc_y} {self.orient}\n")
        
# Generate macro placement file
def gen_macro_cfg(instances=[], fh="macro_placement.cfg"):
    with open(fh, "w") as f:    
        for inst in instances:
            f.write(inst.get_place_str())

# Generate resizer don't touch file
def gen_resizer_no_touch(instances=[], fh="resizer_dont_touch.tcl"):
    cell_names = []
    for inst in instances:
         if("DFE" not in inst.inst_name ):
            cell_names.append(f"{inst.inst_name}")
    with open(fh, "w") as f:    
        f.write("set ::env(RSZ_DONT_TOUCH) {")
        f.write(" ".join(cell_names))
        f.write("}")

# Edit this function if the delay-line element changes
# - Return the hierarchical name for the ith DL element
def dl_inst_name_fmt(i):
    return f"tdc_inst.dl_inst.dl_genblk.dl.rca_genblk\\[{i}\\].FA"

# This shouldn't change for new DL elements
# - Return the hierarchical name for the ith capture register
def capt_reg_inst_name_fmt(i):
    return f"tdc_inst.dl_capt.capt_genblk\\[{i}\\].DFE"

def zeros_inst_fmt(i):
    return f"tdc_inst.dl_inst.dl_genblk.dl.zeros.const_zeros_genblk\\[{i}\\].const_zero"

def ones_inst_fmt(i):
    return f"tdc_inst.dl_inst.dl_genblk.dl.ones.const_ones_genblk\\[{i}\\].const_one"

In [6]:
# Where the DL starts (offset from bottom if using down=1)   
dl_loc_init = (41.4+2*1.38, 204.0)

# DL length and direction
tdc_len = 64
up = False

# Skywater 130A std. cell geometry parameters    
grid_space_x = 0.460
std_cell_height = 2.720

# TT tile dimensions from config
tt_x=161.0
tt_y=225.76

# Leave some space around die perimeter where the DL won't enter
buff_x = grid_space_x*64
buff_y = std_cell_height*5

max_x = 161.0 - buff_x
max_y = 225.76 - buff_y
min_x = buff_x
min_y = buff_y

# Width of cells in delay line
# - Found in $PDK_ROOT/sky130A/libs.ref/sky130_fd_sc_hd/lef/sky130_fd_sc_hd.lef
# - After selecting macro fill in the delay line element dimensions below

# Ex. sky130_fd_sc_hd__fa_4
cell_width_um_dle = 8.280
cell_width_grid_dle = int(cell_width_um_dle/grid_space_x)

# Capt register used is sky130_fd_sc_hd__edfxtp_1
cell_width_um_capt_reg = 11.040
cell_width_grid_capt_reg = int(cell_width_um_capt_reg/grid_space_x)

# Const value cell used is sky130_fd_sc_hd__conb_1
cell_width_um_const = 1.380
cell_width_grid_const = int(cell_width_um_const/grid_space_x)

cell_width_um_decap = 2.760
cell_width_grid_decap = int(cell_width_um_decap/grid_space_x)

# Generate std cell placement configuration
dl_loc      = dl_loc_init
insts       = []
cell_orient = "N"
vspace      = 0
hspace      = 8*cell_width_grid_const

for i in range(tdc_len):
    # Get hierarchical instance names
    conb_one = ones_inst_fmt(i)
    conb_zero = zeros_inst_fmt(i)
    dl_inst = dl_inst_name_fmt(i)
    capt_reg_inst = capt_reg_inst_name_fmt(i)

    cell_orient_inv = "FS" if cell_orient=="N" else "N"

    # Constrain conb cells
    if(cell_orient == "N"):
        insts.append(StdCellInstance(conb_one, (round(dl_loc[0]-cell_width_um_const,5), dl_loc[1]), cell_orient))
        insts.append(StdCellInstance(conb_zero, (round(dl_loc[0]+cell_width_um_dle,5), dl_loc[1]), cell_orient))
    else:
        insts.append(StdCellInstance(conb_one, (round(dl_loc[0]-cell_width_um_const,5), dl_loc[1]), cell_orient))
        insts.append(StdCellInstance(conb_zero, (round(dl_loc[0]+cell_width_um_dle,5), dl_loc[1]), cell_orient))
    
    # Constrain delay line element placement
    insts.append(StdCellInstance(dl_inst, (dl_loc[0], dl_loc[1]), cell_orient))
    
    # Constrain capture register placement
    cr_pos_x = round(dl_loc[0]+cell_width_um_dle+cell_width_um_const, 5)
    insts.append(StdCellInstance(capt_reg_inst, (cr_pos_x, dl_loc[1]), cell_orient))

    # NEW: place constant value cells
    
    
    # Setup next macro orientation
    if(cell_orient == "N"):
        cell_orient = "FS"
    else:
        cell_orient = "N"
    # Setup next macro position
    if(up):
        dl_loc_y=dl_loc[1]+(vspace+1)*std_cell_height
    else:
        dl_loc_y=dl_loc[1]-(vspace+1)*std_cell_height
    dl_loc_x = dl_loc[0]
    if(dl_loc_y > max_y or dl_loc_y < min_y):
        dl_loc_x=dl_loc[0]+cell_width_um_dle+cell_width_um_capt_reg+grid_space_x*16
        dl_loc_y=dl_loc[1]           
        up = not up
    dl_loc = (round(dl_loc_x, 5), round(dl_loc_y,5))

# Generate macro placement config file
gen_resizer_no_touch(insts)

# Generate resizer no touch
gen_macro_cfg(insts)